In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)


Mounted at /content/drive


In [2]:
import pandas as pd
import tensorflow as tf

In [3]:
filename = "/content/drive/MyDrive/SDLC/news_analysis_project/data/final_news_category_dataset.json"
df = pd.read_json(filename, orient='split')
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
df_3 = df[df.category.isin(df.category.value_counts()[:3].index.values)]
df_3

,category,headline,authors,link,short_description,date
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,Ron Dicker,https://www.huffingtonpost.com/entry/morgan-fr...,"""It is not right to equate horrific incidents ...",2018-05-26
...,...,...,...,...,...,...
200800,WELLNESS,The Sleep Library: 11 Soothing Books For Bedtime,,https://www.huffingtonpost.comhttp://www.oprah...,Do you toss and turn until you finally sit up ...,2012-01-28
200802,WELLNESS,The Benefits of Caring for a Pet,"Rita Altman, R.N., Contributor\nSenior Vice Pr...",https://www.huffingtonpost.com/entry/pets-seni...,"For the young as well as the old, especially i...",2012-01-28
200805,WELLNESS,This Is Only the Beginning: Surprising Advice ...,"Ellie Knaus, Contributor\nAtomic Moms Podcast ...",https://www.huffingtonpost.com/entry/life-tips...,"My great-aunt Ida loves to say, ""This is only ...",2012-01-28
200838,ENTERTAINMENT,"Sundance, Ice-T, and Shades of the American Ra...","Courtney Garcia, Contributor\nI tell stories a...",https://www.huffingtonpost.com/entry/sundance-...,Representation of the collective diaspora has ...,2012-01-28


In [6]:
from sklearn.model_selection import train_test_split
y = df_3['category'].astype('category').cat.codes

sentences_train, sentences_test, y_train, y_test = train_test_split(df_3, y, test_size=0.2, stratify=y, random_state=1000)

In [7]:
words_raw = set([k for i in df_3.short_description.values for j in i.split('.') for k in j.split()])
print(f"Number of unique words in the raw dataset: {len(words_raw)}")

Number of unique words in the raw dataset: 82908


## Tokenization

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer_raw = Tokenizer(num_words=60000)
tokenizer_raw.fit_on_texts(sentences_train['short_description'])

X_train_raw = tokenizer_raw.texts_to_sequences(sentences_train['short_description'])
X_test_raw = tokenizer_raw.texts_to_sequences(sentences_test['short_description'])

## Padding sequences

In [13]:
X_train_raw = pad_sequences(X_train_raw, padding='post', maxlen=maxlen_raw)
X_test_raw = pad_sequences(X_test_raw, padding='post', maxlen=maxlen_raw)

In [20]:
print(sentences_train['short_description'][1])
print(X_train_raw[1, :])

Of course it has a song.
[   32    66   403   287    45    14 22609     6     4 14074   368     3
  8955  6706     5  3448  2201     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [9]:
vocab_size_raw = len(tokenizer_raw.word_index) + 1

## Converting y's to categorical

In [21]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test) 

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad

## Loading pretrained word embeddings

In [56]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-03-21 13:11:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-21 13:11:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-21 13:11:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [57]:
!head -n 1 glove.6B.50d.txt | cut -c-50

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.04445


In [58]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [59]:
embedding_dim = 50
embedding_matrix_raw = create_embedding_matrix('glove.6B.50d.txt', 
                                               tokenizer_raw.word_index,
                                               embedding_dim)

# Custom Tranformer from *tensorflow* book example

In [53]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = Sequential([layers.Dense(dense_dim, activation="relu"),
                                      layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None): 
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)

        proj_input = self.layernorm_1(inputs + attention_output) 
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config() 
        config.update({
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "dense_dim": self.dense_dim,
        })
        return config

In [60]:
# vocab_size = 20000
# embed_dim = 50
num_heads = 2
dense_dim = 32
inputs = tf.keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size_raw, embedding_dim, weights=[embedding_matrix_raw], trainable=False)(inputs)
x = TransformerEncoder(embedding_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 50)          2026950   
                                                                 
 transformer_encoder_1 (Tran  (None, None, 50)         23832     
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 153 

In [61]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3)]
             
history_raw = model.fit(X_train_raw, y_train,
                    epochs=30,
                    validation_data=(X_test_raw, y_test), callbacks=callbacks)


Epoch 1/30
1666/1666 [==============================] - 112s 65ms/step - loss: 0.3399 - accuracy: 0.7774 - val_loss: 0.2617 - val_accuracy: 0.8171
Epoch 2/30
1666/1666 [==============================] - 107s 64ms/step - loss: 0.2725 - accuracy: 0.8171 - val_loss: 0.2540 - val_accuracy: 0.8169
Epoch 3/30
1666/1666 [==============================] - 113s 68ms/step - loss: 0.2628 - accuracy: 0.8238 - val_loss: 0.2565 - val_accuracy: 0.8163
Epoch 4/30
1666/1666 [==============================] - 109s 66ms/step - loss: 0.2567 - accuracy: 0.8261 - val_loss: 0.2597 - val_accuracy: 0.8178
Epoch 5/30
1666/1666 [==============================] - 109s 66ms/step - loss: 0.2536 - accuracy: 0.8271 - val_loss: 0.2468 - val_accuracy: 0.8263
Epoch 6/30
1666/1666 [==============================] - 112s 67ms/step - loss: 0.2505 - accuracy: 0.8293 - val_loss: 0.2444 - val_accuracy: 0.8311
Epoch 7/30
1666/1666 [==============================] - 114s 69ms/step - loss: 0.2475 - accuracy: 0.8309 - val_loss: 0

In [62]:
loss_training_raw, accuracy_training_raw = model.evaluate(X_train_raw, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy_training_raw))
loss_test_raw, accuracy_test_raw = model.evaluate(X_test_raw, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy_test_raw))

Training Accuracy: 0.8516
Testing Accuracy:  0.8325
